# **Algorithmic Methods of Data Mining - Fall 2022**

## **Homework 5: The Marvel Universe!**

**Packages that are used troughout the notebook:**

In [38]:
# For data analysis and manipulation
import pandas as pd
import numpy as np

# For graph representation and algos
import networkx as nx

# Utils
from tqdm import tqdm
import sys
from tabulate import tabulate

# For plotting
import plotly.express as px
import plotly.graph_objects as go

## 1. Data

### Matching hero names
Here we examine and prepare the data so that the hero names match between the two datasets. 

In [2]:
# Load data and strip forward slashes and whitespace
nodes = pd.read_csv('data/nodes.csv').applymap(lambda x: x.rstrip('/').strip())
edges = pd.read_csv('data/edges.csv').applymap(lambda x: x.rstrip('/').strip())
network = pd.read_csv('data/hero-network.csv').applymap(lambda x: x.rstrip('/').strip())

In [3]:
# Unique heroes in each dataframe
node_heroes = set(nodes[nodes['type'] == 'hero']['node'])
edge_heroes = set(edges['hero'])
network_heroes = set(pd.concat([network['hero1'], network['hero2']]))

# Number of unique heroes in each dataframe
len(node_heroes), len(edge_heroes), len(network_heroes)

(6439, 6439, 6421)

We would like to have each hero in edges and network represented in the nodes dataframe. So let's see which ones do not have a match in the nodes dataframe.

In [4]:
print('Heroes in edges but not in nodes:', edge_heroes - node_heroes)
print('Heroes in nodes but not in edges:', node_heroes - edge_heroes)

Heroes in edges but not in nodes: {'SPIDER-MAN/PETER PARKER'}
Heroes in nodes but not in edges: {'SPIDER-MAN/PETER PARKERKER'}


In [5]:
# Fix spiderman's name in nodes
nodes.loc[nodes['node'] == 'SPIDER-MAN/PETER PARKERKER', 'node'] = 'SPIDER-MAN/PETER PARKER'

# Update the heroe set for nodes
node_heroes = set(nodes[nodes['type'] == 'hero']['node'])

# Now let's check the network dataframe
print('Heroes in network but not in nodes:', network_heroes - node_heroes)

Heroes in network but not in nodes: {'SPIDER-MAN/PETER PAR'}


In [6]:
# Fix spiderman's name in network
network = network.applymap(lambda x: 'SPIDER-MAN/PETER PARKER' if x == 'SPIDER-MAN/PETER PAR' else x)
network_heroes = set(pd.concat([network['hero1'], network['hero2']]))

# Now we have every hero in the nodes dataframe
print('Number of heroes in nodes but not in network:', len(node_heroes - network_heroes))

Number of heroes in nodes but not in network: 18


We have processed the data such that all the heroes exist in the nodes df. Now let's drop the rows in the network df which have the same hero pair.

In [7]:
# drop same hero pairs
network = network[network['hero1'] != network['hero2']].reset_index(drop=True)

# Are the comics in the nodes and edges matching?
nodes_comics = set(nodes[nodes['type'] == 'comic']['node'])
edges_comics = set(edges['comic'])
len(nodes_comics), len(edges_comics), len(nodes_comics ^ edges_comics) # The comics are matching

(12651, 12651, 0)

Also, we have discovered that there are some heroes and comics that share the same name. We will add "_c" to the comic name so that it will have its own unique node for the upcoming analysis.

In [8]:
iffy_comics = nodes.loc[nodes['node'].duplicated(keep=False) & (nodes['type'] == 'comic'), 'node']
iffy_comics

2078     BLADE
13362    REBEL
13704    SABRE
Name: node, dtype: object

In [9]:
nodes.loc[nodes['node'].isin(iffy_comics) & (nodes['type'] == 'comic'), 'node'] += '_c'
edges.loc[edges['comic'].isin(iffy_comics), 'comic'] += '_c'

In [10]:
from itertools import combinations
from tqdm import tqdm

# Here we create are own network dataframe
my_network = []

for comic, heroes in tqdm(edges.groupby('comic')['hero']):
    heroes = sorted(heroes)
    for combo in combinations(heroes, 2):

        my_network.append({'hero1': combo[0], 'hero2': combo[1], 'comic': comic})

my_network = pd.DataFrame(my_network)
my_network.shape, network.shape

100%|██████████| 12651/12651 [00:01<00:00, 7702.02it/s] 


((579171, 3), (572235, 2))

Lastly, we have discovered that there are some heroes and comics that share the same name. We will add "_c" to the comic name so that it will have its own unique node for the upcoming analysis.

In [11]:
# Delete unnecessary variables
del combo, comic, edge_heroes, edges_comics, heroes, iffy_comics, network_heroes, node_heroes, nodes_comics

## 2. Backend Implementation
We start by creating a list of heroes and their number of appearances so that we can get the top N heroes when needed.

In [12]:
heroes = edges.groupby('hero').size().sort_values(ascending=False)
heroes.head(5)

hero
SPIDER-MAN/PETER PARKER    1577
CAPTAIN AMERICA            1334
IRON MAN/TONY STARK        1150
THING/BENJAMIN J. GR        963
THOR/DR. DONALD BLAK        956
dtype: int64

#### Creating the **First Graph**

We have a bit of an interesting idea to readjust the weights of the edges. You have asked us to have a lower weight/cost for heroes that appear together more frequently. We will add a **small twist** to this idea. These weights will be adjusted based on the number of appearances of the heroes. We will use the following formula to calculate the weight of an edge:
$$
w_{ij} = \frac{|c \in C: h_i \in c| + |c \in C: h_j \in c|}{|c \in C: h_i, h_j \in c|}
$$
where $C$ is the set of all comics, $h_i$ and $h_j$ are two arbitrary heroes in the Marvel Universe. In other words, weight $w_{ij}$ is the inverse of the probability that the two heroes appear together.

In [13]:
edges_weighted = my_network.groupby(['hero1', 'hero2']).size().reset_index().sort_values(by=0, ascending=False)
edges_weighted.columns = ['hero1', 'hero2', 'colab']

edges_weighted['tot'] = edges_weighted['hero1'].map(heroes) + edges_weighted['hero2'].map(heroes)
edges_weighted['weight'] = edges_weighted['tot'] / edges_weighted['colab']
edges_weighted.head(5)

,hero1,hero2,colab,tot,weight
106165,HUMAN TORCH/JOHNNY S,THING/BENJAMIN J. GR,724,1849,2.553867
105742,HUMAN TORCH/JOHNNY S,MR. FANTASTIC/REED R,694,1740,2.507205
141154,MR. FANTASTIC/REED R,THING/BENJAMIN J. GR,690,1817,2.633333
109418,INVISIBLE WOMAN/SUE,MR. FANTASTIC/REED R,682,1616,2.369501
105475,HUMAN TORCH/JOHNNY S,INVISIBLE WOMAN/SUE,675,1648,2.441481


In [14]:
G1 = nx.from_pandas_edgelist(edges_weighted, 'hero1', 'hero2', 'weight')
G1.name = 'First Graph'
G1.number_of_nodes(), G1.number_of_edges(), pd.concat([edges_weighted['hero1'], edges_weighted['hero2']]).nunique(), edges_weighted.shape[0]

(6421, 171644, 6421, 171644)

#### Creating the **Second Graph**

In [15]:
G2 = nx.from_pandas_edgelist(edges, 'hero', 'comic')
G2.name = 'Second Graph'
G2.number_of_nodes(), G2.number_of_edges(), nodes.shape[0], edges.shape[0]

(19090, 96104, 19090, 96104)

In [16]:
G2.add_nodes_from(nodes[nodes['type'] == 'hero']['node'], type='hero')
G2.add_nodes_from(nodes[nodes['type'] == 'comic']['node'], type='comic')
G2.number_of_nodes(), G2.number_of_edges(), nodes.shape[0], edges.shape[0]

(19090, 96104, 19090, 96104)

In [17]:
# Global variable to use inside functinos
COMICS = [node for node in G2.nodes if G2.nodes[node]['type'] == 'comic']

heroes[:10].index[0] in nx.neighbors(G2, COMICS[0])
comics10 = [node for node in G2.nodes if G2.nodes[node]['type'] == 'comic' and len(set(nx.neighbors(G2, node)) & set(heroes[:10].index)) > 0]
len(comics10), len(COMICS)

(6049, 12651)

### Functionality 1 - extract the graph's features

In [65]:
def graph_summary(G: nx.Graph, N: int = 10, verbose = True):
    """Prints some basic features of the graph.
    Args:
        G (nx.Graph): The graph.
        type (int): The type of graph. Can be 1 or 2.
        N (int): Denotes the top N heroes to consider.
    """
    
    if G.name not in ['First Graph', 'Second Graph']:
        ValueError('The graph name is not valid. It must be either "First Graph" or "Second Graph".')

    if verbose:
        print(f'Extracting features for the {G.name}...')
    subnodes = heroes.head(N).index

    # Adding comic nodes if Graph 2
    if G.name == 'Second Graph':
        comics = [node for node in G2.nodes if 
            G2.nodes[node]['type'] == 'comic' and                   # if the node type is comic
            len(set(nx.neighbors(G2, node)) & set(subnodes)) > 0]   # and the comic contains one of the top N heroes
        
        subnodes = [*subnodes, *comics]                             # Add comic nodes to subnodes 
        
    # Create subgraph
    subg = G.subgraph(subnodes)

    num_nodes = subg.number_of_nodes()
    num_edges = subg.number_of_edges()
    nwdensity = nx.density(subg)
    avedegree = np.mean(list(dict(subg.degree()).values()))
    is_sparse = nwdensity < 0.1

    q95 = np.quantile(list(dict(subg.degree()).values()), 0.99)
    hubnodes = [node for node, degree in subg.degree() if degree >= q95]

    info = {
        'Number of nodes': num_nodes,
        'Number of collaborations': num_edges,
        'Network density': nwdensity,
        'Average degree': avedegree,
        'Hub nodes': hubnodes,
        'Sparsity': is_sparse,
        'Degree distribution': dict(subg.degree())
    }

    if G.name == 'First Graph':
        info['Graph Specific Plot Data'] = dict(subg.degree(subnodes))
        info['Graph Specific Plot Name'] = 'Number of collaborations for each hero'
    else:
        info['Graph Specific Plot Data'] = dict(subg.degree(comics))
        info['Graph Specific Plot Name'] = 'Number of heroes in each comic'


    # Reporting values if verbose
    if not verbose:
        return info

    print('Number of nodes:', num_nodes)
    print(f'Network density: {nwdensity:.4f}')
    print(f'Average degree: {avedegree:.4f}')

    print('Hub nodes:', '\t\t'.join(hubnodes))
    print('The network is sparse:', is_sparse)

    print('Network degree distribution:')
    print(pd.DataFrame(dict(subg.degree()).values()).describe().T)

    return info

### Functionality 2 - Find top superheroes!

In [95]:
def top_heroes(G: nx.Graph, node: str, metric: int, N: int = 10, verbose = True):
    """Prints the top N heroes based on the given metric.
    Args:
        G (nx.Graph): The graph.
        node (str): The node (hero or comic).
        metric (int): Integer denoting the metric. Can be 1, 2, 3 or 4, that corresponds to:
            1: Betweeness
            2: PageRank
            3: ClosenessCentrality
            4: DegreeCentrality
        N (int): Denotes the top N heroes to consider.
    """
    if metric not in [1, 2, 3, 4]:
        raise ValueError('metric must be 1, 2, 3 or 4')
    
    measure = ['Betweeness', 'PageRank', 'ClosenessCentrality', 'DegreeCentrality']
    subnodes = heroes[:N].index
    subg = G.subgraph(subnodes)

    if metric == 1:
        res = nx.betweenness_centrality(subg, normalized=True, weight='weight')
        print(f'{node}\'s {measure[metric-1]}: {res[node]:.4f}') if verbose else None
    elif metric == 2:
        res = nx.pagerank(G, weight='weight')
        print(f'{node}\'s {measure[metric-1]}: {res[node]:.4f}') if verbose else None
    elif metric == 3:
        res = nx.closeness_centrality(G, u=node)
        print(f'{node}\'s {measure[metric-1]}: {res:.4f}') if verbose else None
    else:
        res = nx.degree_centrality(G)
        print(f'{node}\'s {measure[metric-1]}: {res[node]:.4f}') if verbose else None

    # print(res)
    if metric == 3:
        info = {
            'Metric of node': res,
            'Name of metric': measure[metric-1],
        }
    else:
        info = {
            'Average metric': np.mean(list(res.values())),
            'Metric of node': res[node],
            'Name of metric': measure[metric-1],
        }


    return info

# top_heroes(G1, heroes.index[1], 1, 50)
# top_heroes(G1, heroes.index[1], 2, 50)
# top_heroes(G1, heroes.index[1], 3, 50)
top_heroes(G1, heroes.index[1], 4, 50)

CAPTAIN AMERICA's DegreeCentrality: 0.2989


{'Average metric': 0.008327620478172043,
 'Metric of node': 0.29890965732087227,
 'Name of metric': 'DegreeCentrality'}

### Functionality 3 - Shortest ordered Route

In [236]:
def shortest_route(G: nx.Graph, hero_seq: list, N: int = 10, verbose = True):
    """Prints the shortest route from the first hero to the last hero in the list.
    Args:
        G (nx.Graph): The graph.
        hero_seq (list): The sequence of heroes to visit.
        N (int): Denotes the top N heroes to consider.
    """
    if G.name not in ['Second Graph']:
        raise ValueError('Graph must be the second graph')

    subheroes = heroes[:N].index

    # Create list of comics by checking the node type and if it has a neighbor that belongs to the top N heroes
    comics = [node for node in G2.nodes if G2.nodes[node]['type'] == 'comic' and len(set(G2.neighbors(node)).intersection(set(subheroes))) > 0]

    # Create subgraph
    subg = G.subgraph([*subheroes, *comics])

    # Check if things are going well
    print(G) if verbose else None

    # Check if the sequence of heroes are in the top N heroes
    for hero in hero_seq:
        if hero not in subg.nodes:
            print(f'A hero in the sequence is not a top {N} hero. Your lame hero is {hero}. Please try again with more popular heroes or a larger N.')
            print('Heroes must be one of the following:', *subg.nodes)
            return

    paths = []
    for i in range(len(subheroes)-1):
        paths.append(nx.shortest_path(subg, source=subheroes[i], target=subheroes[i+1]))

    print('Shortest route:', *paths)

    info = {
        'Shortest route': paths,
        'Subgraph': subg,
        'Hero sequence': hero_seq
    }

    return info
# TODO: There could be cases with no paths find those :/
info = shortest_route(G2, heroes[:5].index, 10)

Graph named 'Second Graph' with 19090 nodes and 96104 edges
Shortest route: ['SPIDER-MAN/PETER PARKER', 'A 11', 'CAPTAIN AMERICA'] ['CAPTAIN AMERICA', "A '00", 'IRON MAN/TONY STARK'] ['IRON MAN/TONY STARK', 'A 1', 'THING/BENJAMIN J. GR'] ['THING/BENJAMIN J. GR', 'A 1', 'THOR/DR. DONALD BLAK'] ['THOR/DR. DONALD BLAK', 'A 1', 'HUMAN TORCH/JOHNNY S'] ['HUMAN TORCH/JOHNNY S', 'A 1', 'MR. FANTASTIC/REED R'] ['MR. FANTASTIC/REED R', 'A 1', 'HULK/DR. ROBERT BRUC'] ['HULK/DR. ROBERT BRUC', 'A3 10', 'WOLVERINE/LOGAN'] ['WOLVERINE/LOGAN', 'A3 10', 'INVISIBLE WOMAN/SUE']


### Functionality 4 - Disconnecting Graphs

In [290]:
from networkx.algorithms.connectivity.utils import build_auxiliary_edge_connectivity, build_auxiliary_node_connectivity
from networkx.algorithms.flow import build_residual_network, edmonds_karp
from networkx.algorithms.connectivity.cuts import minimum_st_edge_cut



In [299]:
# Functionality 4 - Disconnecting Graphs
# Input:

# The graph data
# heroA: a superhero to which will relate sub-graph G_a
# heroB: a superhero to which will relate sub-graph G_b
# N: denoting the top N heroes that their data should be considered
# Output:

# The minimum number of links (by considering their weights) required to disconnect the original graph in two disconnected subgraphs: G_a and G_b.

def disconnecting_graphs(G: nx.Graph, heroA: str, heroB: str, N: int = 10, verbose: bool = False) -> dict:
    """Prints the minimum number of links required to disconnect the original graph in two disconnected subgraphs.
    Args:
        G (nx.Graph): The graph.
        heroA (str): The first hero.
        heroB (str): The second hero.
        N (int): Denotes the top N heroes to consider.
    """

    subg = G.subgraph(heroes.index[:N])
    
    cutset = nx.minimum_edge_cut(subg, heroA, heroB)
    # cutset = minimum_edge_cut(subg, heroA, heroB)
    print(f'Number of links to cut to disconnect {heroA} and {heroB}: {len(cutset)}') if verbose else None

    # Check if path exists after cutting the links
    alteredg = subg.copy()
    alteredg.remove_edges_from(cutset)
    print(f'Path exists between {heroA} and {heroB}: {nx.has_path(alteredg, heroA, heroB)}') if verbose else None

    info = {'Number of links to cut': len(cutset),
            'Cutset': cutset,
            'Subgraph': subg,
            'HeroA': heroA,
            'HeroB': heroB
    }

    return info
    
info = disconnecting_graphs(G1, heroes.index[100], heroes.index[200], 300, verbose=True)
None

Number of links to cut to disconnect ZABU and ROM, SPACEKNIGHT: 106
Path exists between ZABU and ROM, SPACEKNIGHT: False


### Functionality 5 - Extracting Communities

In [364]:
from itertools import combinations

def extracting_communities(G: nx.Graph, N: int = 10, hero1: str = None, hero2: str = None, verbose: bool = True, method: str = 'girvan_newman') -> dict:
    """Prints the minimum number of edges that should be removed to form communities and a list of communities.
    Args:
        G (nx.Graph): The graph.
        N (int): Denotes the top N heroes to consider.
        hero1 (str): The first hero.
        hero2 (str): The second hero.
        method (str): The method to use for extracting communities. Possible values are 'girvan_newman', 'asyn_lpa_communities', 
        'asyn_fluidc' and 'label_propagation_communities'
    """
    subnodes = heroes.index[:N]
    subg = G.subgraph(subnodes)

    if method == 'girvan_newman':
        res = nx.algorithms.community.centrality.girvan_newman(subg)
        communities = next(res)
    elif method == 'asyn_lpa_communities':
        res = nx.algorithms.community.asyn_lpa_communities(subg)
    elif method == 'asyn_fluidc':
        res = nx.algorithms.community.asyn_fluidc(subg, k=3)
    elif method == 'label_propagation_communities':
        res = nx.algorithms.community.label_propagation_communities(subg)
        communities = res
    else:
        print('Invalid method. Please choose one of the following: girvan_newman, asyn_lpa_communities, asyn_fluidc, label_propagation_communities')
        return

    print(res)

    path_exists = False
    for community in communities:
        if hero1 in community and hero2 in community:
            path_exists = True
            break

    num_edges_between_communities = 0
    edges_removed = []
    for community1, community2 in combinations(communities, 2):
        for node1 in community1:
            for node2 in community2:
                if subg.has_edge(node1, node2):
                    num_edges_between_communities += 1
                    edges_removed.append((node1, node2))
                    edges_removed.append((node2, node1))
            

    info = {
        'Communities': communities,
        'Subgraph': subg,
        'Hero1': hero1,
        'Hero2': hero2,
        'Same community': path_exists,
        'Edges to cut': num_edges_between_communities,
        'Edges removed': edges_removed
    }

    if not verbose:
        return info


    print('Communities:') 
    for i, community in enumerate(communities):
        print(f'Community {i+1}:', *community)

    print(f'Path exists between {hero1} and {hero2}: {path_exists}')
    print(f'Number of edges to cut to form communities: {num_edges_between_communities}')
    return info


extracting_communities(G1, 100, method='girvan_newman')
None

<generator object girvan_newman at 0x7fa2ffb30c80>
Communities:
Community 1: SUMMERS, NATHAN CHRI IRON MAN IV/JAMES R. BOOMER/TABITHA SMITH NORRISS, SISTER BARB NELSON, FRANKLIN FOG CAPTAIN MARVEL II/MO WASP/JANET VAN DYNE QUICKSILVER/PIETRO M ICEMAN/ROBERT BOBBY NIGHTCRAWLER/KURT WA GAMBIT/REMY LEBEAU CAPTAIN AMERICA THOMPSON, EUGENE FLA ROGUE ANT-MAN/DR. HENRY J. MASTERS, ALICIA REIS THING/BENJAMIN J. GR PARKER, MAY WOLFSBANE/RAHNE SINC DR. DOOM/VICTOR VON STORM/ORORO MUNROE S IRON MAN/TONY STARK SUNSPOT/ROBERTO DACO MARVEL GIRL/JEAN GRE HUMAN TORCH/JOHNNY S DUGAN, TIMOTHY ALOYI COLOSSUS II/PETER RA MACTAGGERT, MOIRA KI SILVER SURFER/NORRIN BALDER [ASGARDIAN] HAVOK/ALEX SUMMERS USAGENT/CAPTAIN JOHN NOVA/RICHARD RIDER HULK/DR. ROBERT BRUC VOLSTAGG HOGUN [ASGARDIAN] POLARIS/LORNA DANE KINGPIN/WILSON FISK HAWK CAGE, LUKE/CARL LUCA DR. STRANGE/STEPHEN PAGE, KAREN LEEDS, BETTY BRANT JUBILEE/JUBILATION L JAMESON, J. JONAH WATSON-PARKER, MARY HERCULES [GREEK GOD] FURY, COL. NICHOLAS PSYLOCK

## 3. Frontend Implementation

### Visualization 1 - Visualize some features of the network

In [ ]:
info = graph_summary(G2, N=300, verbose=False)
print(tabulate(pd.DataFrame.from_dict(info, orient='index', columns=['Value']).T[['Number of nodes', 'Number of collaborations', 'Network density', 'Average degree', 'Sparsity']], headers='keys', tablefmt='psql'))
# This the table

# Hub nodes
print('Hub nodes:', '\t'.join(info['Hub nodes'][:10]))

# Histogram of the degree distribution with px, logaritmic scale
fig = px.histogram(pd.DataFrame.from_dict(info['Degree distribution'], orient='index', columns=['Degree']), x='Degree', log_y=True)
fig.update_layout(title='Degree distribution', yaxis_title='Count')
fig.show()

# Graph specific plot
plot_data = pd.DataFrame.from_dict(info['Graph Specific Plot Data'], orient='index', columns=['Degree']).sort_values(by='Degree', ascending=False)
plot_data = plot_data[:50] # Limiting to 50 for better visualization
plot_data.sort_values(by='Degree', ascending=True, inplace=True)
fig = px.bar(plot_data, x='Degree', y=plot_data.index, orientation='h')
fig.update_layout(title=info['Graph Specific Plot Name'], yaxis_title='')
fig.show()

### Visualization 2 - Visualize centrality measure

In [98]:
info = top_heroes(G1, heroes.index[1], 1, 50, verbose=False)
print(tabulate(pd.DataFrame.from_dict(info, orient='index'), headers=['Value'], tablefmt='fancy_grid'))

╒════════════════╤══════════════════════╕
│                │ Value                │
╞════════════════╪══════════════════════╡
│ Average metric │ 0.029676870748299323 │
├────────────────┼──────────────────────┤
│ Metric of node │ 0.179421768707483    │
├────────────────┼──────────────────────┤
│ Name of metric │ Betweeness           │
╘════════════════╧══════════════════════╛


### Visualization 3 - Visualize the shortest-ordered route

In [393]:
info = shortest_route(G2, heroes[:5].index, 10, verbose=False)

comics = [t[1] for t in info['Shortest route']]
route_df = pd.DataFrame(comics, columns=['Comic Name'], index=[f'Comic {i+1}' for i in range(len(comics))]).T
print(tabulate(route_df, headers='keys', tablefmt='psql'))

nodes_in_sr = list(np.unique([node for path in info['Shortest route'] for node in path]))
edges_in_sr = [[(x[0], x[1]), (x[1], x[2]), (x[1], x[0]), (x[2], x[1])] for x in info['Shortest route']]
# sort each tuple in the list
edges_in_sr = [item for sublist in edges_in_sr for item in sublist]

subg = info['Subgraph']
subg = subg.subgraph(nodes_in_sr)

# for edges in edges_in_sr make path = True, otherwise make path = False
for edge in subg.edges():
    if edge in edges_in_sr:
        subg.edges[edge]['path'] = True
    else:
        subg.edges[edge]['path'] = False

for hero in info['Hero sequence']:
    subg.nodes[hero]['seq'] = 'seq'

# Now we implement the example for our graph subg
pos = nx.spring_layout(subg)

edges = {'edge_x': [],'edge_y': []}
colored_edges = {'edge_x': [],'edge_y': []}

for edge in subg.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    if subg.edges[edge]['path'] == True:
        colored_edges['edge_x'].append(x0)
        colored_edges['edge_x'].append(x1)
        colored_edges['edge_x'].append(None)
        colored_edges['edge_y'].append(y0)
        colored_edges['edge_y'].append(y1)
        colored_edges['edge_y'].append(None)
    else:
        edges['edge_x'].append(x0)
        edges['edge_x'].append(x1)
        edges['edge_x'].append(None)
        edges['edge_y'].append(y0)
        edges['edge_y'].append(y1)
        edges['edge_y'].append(None)


edge_trace = go.Scatter(
    x=edges['edge_x'], y=edges['edge_y'],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

colored_edge_trace = go.Scatter(
    x=colored_edges['edge_x'], y=colored_edges['edge_y'],
    line=dict(width=2, color='red'),
    hoverinfo='none',
    mode='lines')

nodes = {'node_x': [],'node_y': []}
for node in subg.nodes():
    x, y = pos[node]
    nodes['node_x'].append(x)
    nodes['node_y'].append(y)


print('Im done with the node positions')

node_trace = go.Scatter(
    x=nodes['node_x'], y=nodes['node_y'],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        line_width=2))

colors = {'hero': '#F3A5AA', 'comic': '#447BBE', 'seq': '#DF1F2D'}

node_trace.marker.color = [colors[subg.nodes[node]['seq'] if 'seq' in subg.nodes[node] else subg.nodes[node]['type']] for node in subg.nodes()]
node_trace.text = [f'{node}: {subg.nodes[node]["type"]}' for node in subg.nodes()]

print('Im done with node adjacencies and node text')

fig = go.Figure(data=[edge_trace, colored_edge_trace, node_trace],
             layout=go.Layout(
                title='Shortest Route',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text=f"Sequence of heroes: {', '.join(info['Hero sequence'])}",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

Shortest route: ['SPIDER-MAN/PETER PARKER', 'A 11', 'CAPTAIN AMERICA'] ['CAPTAIN AMERICA', "A '00", 'IRON MAN/TONY STARK'] ['IRON MAN/TONY STARK', 'A 1', 'THING/BENJAMIN J. GR'] ['THING/BENJAMIN J. GR', 'A 1', 'THOR/DR. DONALD BLAK'] ['THOR/DR. DONALD BLAK', 'A 1', 'HUMAN TORCH/JOHNNY S'] ['HUMAN TORCH/JOHNNY S', 'A 1', 'MR. FANTASTIC/REED R'] ['MR. FANTASTIC/REED R', 'A 1', 'HULK/DR. ROBERT BRUC'] ['HULK/DR. ROBERT BRUC', 'A3 10', 'WOLVERINE/LOGAN'] ['WOLVERINE/LOGAN', 'A3 10', 'INVISIBLE WOMAN/SUE']
+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|            | Comic 1   | Comic 2   | Comic 3   | Comic 4   | Comic 5   | Comic 6   | Comic 7   | Comic 8   | Comic 9   |
|------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------|
| Comic Name | A 11      | A '00     | A 1       | A 1       | A 1       | A 1       | A 1       | A3 10     | A3 10     |
+

### Visualization 4 - Visualize the disconnected graph

In [ ]:
# Retrieve info
info = disconnecting_graphs(G1, heroes.index[20], heroes.index[21], 30, verbose=False)

print(f'Number of links to cut to disconnect {heroes.index[100]} and {heroes.index[200]}: {info["Number of links to cut"]}')

original_graph = info['Subgraph']
cutset = info['Cutset']

# We remove the cutset from the original graph
altered_graph = original_graph.copy()
altered_graph.remove_edges_from(cutset)

# We plot the original graph
pos = nx.spring_layout(original_graph)

edges = {'edge_x': [],'edge_y': []}
for edge in original_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edges['edge_x'].append(x0)
    edges['edge_x'].append(x1)
    edges['edge_x'].append(None)
    edges['edge_y'].append(y0)
    edges['edge_y'].append(y1)
    edges['edge_y'].append(None)

edge_trace = go.Scatter(
    x=edges['edge_x'], y=edges['edge_y'],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

nodes = {'node_x': [],'node_y': []}
for node in original_graph.nodes():
    x, y = pos[node]
    nodes['node_x'].append(x)
    nodes['node_y'].append(y)

node_trace = go.Scatter(
    x=nodes['node_x'], y=nodes['node_y'],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        line_width=2))
        
# Red if heroa or herob, else blue
node_trace.marker.color = ['red' if node in [info['HeroA'], info['HeroB']] else 'blue' for node in original_graph.nodes()]
# camelcase for heroname
node_trace.text = [node.split('/')[0].title() for node in original_graph.nodes()]

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Original Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40))
                    )
fig.show()

pos = nx.spring_layout(altered_graph)

# Remove edges from the original graph
edges = {'edge_x': [],'edge_y': []}
for edge in altered_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edges['edge_x'].append(x0)
    edges['edge_x'].append(x1)
    edges['edge_x'].append(None)
    edges['edge_y'].append(y0)
    edges['edge_y'].append(y1)
    edges['edge_y'].append(None)

edge_trace = go.Scatter(
    x=edges['edge_x'], y=edges['edge_y'],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

max_weight = max([edge[2]['weight'] for edge in original_graph.edges(data=True)]) + 0.1
colorpalette = px.colors.sequential.BuPu
m = len(colorpalette)
# edge_trace.line.color = [colorpalette[int(edge[2]['weight']/max_weight*m)] for edge in altered_graph.edges(data=True)]

nodes = {'node_x': [],'node_y': []}
for node in altered_graph.nodes():
    x, y = pos[node]
    nodes['node_x'].append(x)
    nodes['node_y'].append(y)

node_trace = go.Scatter(
    x=nodes['node_x'], y=nodes['node_y'],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        line_width=2))

# Red if heroa or herob, else blue
node_trace.marker.color = ['red' if node in [info['HeroA'], info['HeroB']] else 'blue' for node in altered_graph.nodes()]
# camelcase for heroname
node_trace.text = [node.split('/')[0].title() for node in altered_graph.nodes()]

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Altered Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40))
                    )
fig.show()
    

### Visualization 5 - Visualize the communities

In [398]:
info = extracting_communities(G1, 30, 'CAPTAIN AMERICA', 'IRON MAN/TONY STARK', verbose=False)

# Information to print
print('The number of edges to cut is:', info['Edges to cut'])
print('Captain America and Ironman belong to the same community:', info['Same community'])
for i, community in enumerate(info['Communities']):
    print(f'Community {i+1}:')
    print(tabulate([community], tablefmt='fancy_grid'))

# We plot the communities
subg = info['Subgraph']
pos = nx.spring_layout(subg)

edges = {'edge_x': [],'edge_y': []}
edges_removed = {'edge_x': [],'edge_y': []}
for edge in subg.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    if edge in info['Edges removed']:
        edges_removed['edge_x'].append(x0)
        edges_removed['edge_x'].append(x1)
        edges_removed['edge_x'].append(None)
        edges_removed['edge_y'].append(y0)
        edges_removed['edge_y'].append(y1)
        edges_removed['edge_y'].append(None)
    else:
        edges['edge_x'].append(x0)
        edges['edge_x'].append(x1)
        edges['edge_x'].append(None)
        edges['edge_y'].append(y0)
        edges['edge_y'].append(y1)
        edges['edge_y'].append(None)

edge_trace = go.Scatter(
    x=edges['edge_x'], y=edges['edge_y'],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

edge_trace_removed = go.Scatter(
    x=edges_removed['edge_x'], y=edges_removed['edge_y'],
    line=dict(width=0.5, color='red'),
    hoverinfo='none',
    mode='lines')


nodes = {'node_x': [],'node_y': []}
for node in subg.nodes():
    x, y = pos[node]
    nodes['node_x'].append(x)
    nodes['node_y'].append(y)

node_trace = go.Scatter(
    x=nodes['node_x'], y=nodes['node_y'],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        line_width=2))

colorpalette = px.colors.sequential.Plasma
def get_color(node):
    for i, community in enumerate(info['Communities']):
        if node in community:
            if i == 0:
                return 'red'
            return colorpalette[i]

node_trace.marker.color = [get_color(node) for node in subg.nodes()]
node_trace.text = [node.split('/')[0].title() for node in subg.nodes()]

fig = go.Figure(data=[edge_trace, edge_trace_removed, node_trace],
                layout=go.Layout(
                    title='Communities',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40))
                    )
fig.show()

<generator object girvan_newman at 0x7fa2ff2ce0b0>
The number of edges to cut is: 29
Captain America and Ironman belong to the same community: True
Community 1:
╒══════════════════════╕
│ THOR/DR. DONALD BLAK │
╘══════════════════════╛
Community 2:
╒══════╤═════════════════════╤═════════════════════╤══════════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤═════════════════════╤══════════════════════╤═════════════════════╤══════════════════════╤═════════════════╤═════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤═════════════════════╤═════════════════════════╤════════╤═════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤══════════════════════╤═════════════════╕
│ HAWK │ DR. STRANGE/STEPHEN │ PROFESSOR X/CHARLES │ BEAST/HENRY &HANK& P │ SCARLET WITCH/WANDA │ JAMESON, J. JONAH │ WASP/JANET VA

## 4. Command Line Questions

In [400]:
%%bash

#1.What is the most popular pair of heroes (often appearing together in the comics)?
echo "Most popular pair of heroes:"
awk 'NR>1{c[$0]++} END{for (k in c)print k,","c[k]}' data/hero-network.csv | sort -k3 -nr -t,  > cl_output/uniq-sort.txt
head -n 1 cl_output/uniq-sort.txt 

#2.Number of comics per hero.
echo "Number of comics per hero:"
cut -d',' -f1 data/edges.csv | awk 'NR>1{c[$0]++} END{for (k in c)print k,","c[k]}' > cl_output/num_com_per_hero.txt
head -n 10 cl_output/num_com_per_hero.txt

#3.Average number of heroes per comic
echo "Average number of heroes per comic:"
sort -u data/edges.csv | cut -d',' -f1 data/edges.csv | awk 'NR>1{c[$0]++} END{for (k in c)print k,","c[k]}' | awk -F',' '{sum+=$2; ++n} END { print "Avg: "sum"/"n"="sum/n }' 

Most popular pair of heroes:
"PATRIOT/JEFF MACE","PATRIOT/JEFF MACE" ,1275
Number of comics per hero:
SODIUM ,2
"MUNROE ,3
ACROBAT/CARL ZANTE ,3
"BAPTISTE ,1
MAN-BULL/BULL TAURUS ,9
SIBERCAT/ ,2
BRUTE/REED RICHARDS ,9
RINGMASTER/MAYNARD T ,34
"MARLOPOLIS ,1
MASTER MAN II/ALEXAN ,2
Average number of heroes per comic:
Avg: 96104/5777=16.6356


## 5. Bonus - PageRank on MapReduce

## 6. Algorithmic Question